In [30]:
import json
import os
import joblib

## Extract features for First Model

In [31]:
import spacy
from spacy.tokens import Doc

nlp = spacy.load("en_core_web_md")

def extract_features(sentence, predicate_mask):
    """
    Extracts features for each token in the sentence based on the predicate_mask.

    Args:
        sentence (list): List of words in the sentence.
        predicate_mask(list): List of 0s and a single 1 indicating position of predicate

    Returns:
        list: A list of dictionaries, where each dictionary contains features for a token.
    """
    # Convert predicate_mask to predicate_id
    predicate_index = predicate_mask.index(1)
    
    # # Convert predicate_id to 0-based index for Python
    # predicate_index = predicate_id - 1

    # Join the sentence into a single string for SpaCy processing
    raw_doc = Doc(nlp.vocab, words=sentence)
    doc = nlp(raw_doc)
    
    # Initialize a list to store features for each token
    features = []

    # Get the predicate token and its lemma
    predicate_token = doc[predicate_index]
    predicate_lemma = predicate_token.lemma_  # Use the lemma of the predicate

    # Function to safely traverse the dependency tree and get the path to the predicate
    def get_dependency_path(token, predicate_token):
        """
        Traverses the dependency tree from the token to the predicate token, avoiding cycles.

        Args:
            token (spacy.Token): The starting token.
            predicate_token (spacy.Token): The predicate token.

        Returns:
            list: A list of dependency relations forming the path to the predicate.
        """
        path = []
        current_token = token
        visited = set()  # To avoid cycles

        # Traverse the dependency tree until we reach the predicate or hit a cycle
        while current_token.i != predicate_token.i:
            if current_token.i in visited:
                # If we encounter a cycle, stop traversal
                break
            if current_token.dep_ != "ROOT":
                path.append(current_token.dep_)
            visited.add(current_token.i)
            current_token = current_token.head

        # Add the predicate's lemma to the path
        path.append(predicate_lemma)  # Use the predicate's lemma here
        return path

    # Iterate over each token in the sentence
    for token in doc:
        # Feature 1: Directed dependency path from the token to the predicate + predicate lemma
        dep_path = get_dependency_path(token, predicate_token)
        feature1 = "->".join(dep_path)

        # Feature 2: Part-of-speech tag of the token
        feature2 = token.pos_

        # Feature 3: Relative position of the token to the predicate
        feature3 = token.i - predicate_index

        # Append the features for this token to the list
        features.append({
            "dep_path_predicate": feature1,
            "pos_tag": feature2,
            "relative_position": feature3
        })
    # Debug: Print the number of tokens and features
    if len(sentence) != len(features):
        print(f"Tokenization Mismatch")

    return features

## Load the first model

In [32]:
loaded_data = joblib.load("../assignment_1/srl_new_model.pkl")
model = loaded_data["model"]
scaler = loaded_data["scaler"]
vectorizer = loaded_data["vectorizer"]
label_encoder = loaded_data["label_encoder"]

## Inference Function of First Model

In [33]:
def perform_srl(sentence, predicate_id, model, vectorizer, label_encoder, scaler):
    """
    Performs SRL on a standalone sentence given the predicate.

    Args:
        sentence (list): List of words in the sentence.
        predicate_id (int): 1-based index of the predicate in the sentence.
        model (LogisticRegression): Trained logistic regression model.
        vectorizer (DictVectorizer): Fitted vectorizer.
        label_encoder (LabelEncoder): Fitted label encoder.
        scaler (StandardScaler): Fitted scaler.

    Returns:
        list: Predicted semantic roles for each token in the sentence.
    """
    # Extract features for the sentence
    features = extract_features(sentence, predicate_id)

    # Transform features into a sparse matrix
    X = vectorizer.transform(features)

    # Scale the features
    X_scaled = scaler.transform(X)

    # Make predictions
    y_pred = model.predict(X_scaled)
    y_pred_labels = label_encoder.inverse_transform(y_pred)

    return y_pred_labels

## Function to go through the json dataset and test the model with the data (Also stores the predicted data in a model)

In [47]:
def test_srl_capability_and_save(json_file_path, output_directory="srl_results"):
    """
    Loads a JSON challenge set file, calculates the failure rate for the SRL model,
    adds predicted roles to the data, and saves the results to a new JSON file.

    Args:
        json_file_path (str): The path to the JSON file for a specific capability.
        output_directory (str): The directory where the result JSON files will be saved.

    Returns:
        dict: A dictionary containing the capability name, total test cases,
              total failures, and failure rate, or None if an error occurred.
    """
    os.makedirs(output_directory, exist_ok=True)

    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found at {json_file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {json_file_path}")
        return None

    capability_info = data[0]
    capability_name = capability_info["capability"]
    test_cases = capability_info["test_cases"]
    
    type_1_failure_per_alternation_pair = 0
    type_2_failure_per_alternation_pair = 0
    failures_per_alternation_pair = 0
    processed_test_cases = [] # To store test cases with predictions

    print(f"\n--- Testing Capability: {capability_name} ---")

    for i, test_instance in enumerate(test_cases):
        print(f"\n  Instance {i + 1}:")

        current_instance_failed = False
        processed_instance = test_instance.copy() # Create a copy to add predictions

        # Test the first alternation form
        sentence1_key = list(test_instance.keys())[0]
        predicate_mask1_key = list(test_instance.keys())[1]
        expected_roles1_key = list(test_instance.keys())[2]

        sentence1 = test_instance[sentence1_key]
        predicate_mask1 = test_instance[predicate_mask1_key]
        expected_roles1 = test_instance[expected_roles1_key]

        predicted_roles1 = list(perform_srl(sentence1, predicate_mask1, model, vectorizer, label_encoder, scaler))
        processed_instance[f"{sentence1_key.replace('sentence', 'predicted_roles')}"] = predicted_roles1 # Add predicted roles

        print(f"    Form 1: {' '.join(sentence1)}")
        print(f"      Predicate Mask: {predicate_mask1}")
        print(f"      Expected Roles: {expected_roles1}")
        print(f"      Predicted Roles: {predicted_roles1}")

        if predicted_roles1 != expected_roles1:
            current_instance_failed = True
            type_1_failure_per_alternation_pair += 1
            print("      -> FAILURE for Form 1")
        else:
            print("      -> PASS for Form 1")

        # Test the second alternation form
        sentence2_key = list(test_instance.keys())[3]
        predicate_mask2_key = list(test_instance.keys())[4]
        expected_roles2_key = list(test_instance.keys())[5]

        sentence2 = test_instance[sentence2_key]
        predicate_mask2 = test_instance[predicate_mask2_key]
        expected_roles2 = test_instance[expected_roles2_key]

        predicted_roles2 = list(perform_srl(sentence2, predicate_mask2, model, vectorizer, label_encoder, scaler))
        processed_instance[f"{sentence2_key.replace('sentence', 'predicted_roles')}"] = predicted_roles2 # Add predicted roles

        print(f"    Form 2: {' '.join(sentence2)}")
        print(f"      Predicate Mask: {predicate_mask2}")
        print(f"      Expected Roles: {expected_roles2}")
        print(f"      Predicted Roles: {predicted_roles2}")

        if predicted_roles2 != expected_roles2:
            current_instance_failed = True
            type_2_failure_per_alternation_pair += 1
            print("      -> FAILURE for Form 2")
        else:
            print("      -> PASS for Form 2")

        if current_instance_failed:
            failures_per_alternation_pair += 1
            processed_instance["failure"] = "true"
        else:
            processed_instance["failure"] = "false"
        
        processed_test_cases.append(processed_instance)

    total_alternation_pairs = len(test_cases)
    type_1_failure_rate = (type_1_failure_per_alternation_pair / total_alternation_pairs) * 100 if total_alternation_pairs > 0 else 0
    type_2_failure_rate = (type_2_failure_per_alternation_pair / total_alternation_pairs) * 100 if total_alternation_pairs > 0 else 0
    failure_rate = (failures_per_alternation_pair / total_alternation_pairs) * 100 if total_alternation_pairs > 0 else 0

    # Prepare the output data structure
    output_data = [
        {
            "id": capability_info["id"],
            "capability": capability_name,
            "description": capability_info["description"],
            "total_alternation_pairs": total_alternation_pairs,
            "failures_in_alternation_pairs": failures_per_alternation_pair,
            "type_1_failures": type_1_failure_per_alternation_pair,
            "type_1_failure_rate": f"{type_1_failure_rate:.2f}%",
            "type_2_failures": type_2_failure_per_alternation_pair,
            "type_2_failure_rate": f"{type_2_failure_rate:.2f}%",
            "failure_rate_percent": f"{failure_rate:.2f}%",
            "test_cases": processed_test_cases
        }
    ]

    # Define the output file path
    output_file_name = os.path.basename(json_file_path).replace(".json", "_results.json")
    output_file_path = os.path.join(output_directory, output_file_name)

    # Save the results to a new JSON file
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        json.dump(output_data, outfile, indent=2, ensure_ascii=False)
    print(f"\nResults saved to: {output_file_path}")

    return {
        "capability": capability_name,
        "total_alternation_pairs": total_alternation_pairs,
        "failures_in_alternation_pairs": failures_per_alternation_pair,
        "failure_rate_percent": f"{failure_rate:.2f}%"
    }

In [48]:
json_files = [
    "active_passive_voice_alternation.json",
    "dative_alternation.json",
    "locative_alternation.json",
    "causative_inchoative_alternation.json",
    "temporal_placement_alternation.json"
]


summary_results = []
output_dir = "first_srl_model_results" # Directory to save new JSONs

print(f"Saving results to directory: {output_dir}")
for json_file in json_files:
    result = test_srl_capability_and_save(json_file, output_dir)
    if result:
        summary_results.append(result)

Saving results to directory: first_srl_model_results

--- Testing Capability: Active/Passive Voice Alternation ---

  Instance 1:
    Form 1: The dog chased the ball .
      Predicate Mask: [0, 0, 1, 0, 0, 0]
      Expected Roles: ['O', 'ARG0', 'V', 'O', 'ARG1', 'O']
      Predicted Roles: [np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('ARG1'), np.str_('O')]
      -> FAILURE for Form 1
    Form 2: The ball was chased by the dog .
      Predicate Mask: [0, 0, 0, 1, 0, 0, 0, 0]
      Expected Roles: ['O', 'ARG1', 'O', 'V', 'O', 'O', 'ARG0', 'O']
      Predicted Roles: [np.str_('O'), np.str_('ARG1'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O')]
      -> FAILURE for Form 2

  Instance 2:
    Form 1: Anna loves Benjamin .
      Predicate Mask: [0, 1, 0, 0]
      Expected Roles: ['ARG0', 'V', 'ARG1', 'O']
      Predicted Roles: [np.str_('ARG0'), np.str_('V'), np.str_('ARG1'), np.str_('O')]
      -> PASS for Form 1
    Form 2: Benjamin 

In [49]:
print("\n--- Overall Summary of All Capability Tests ---")
for r in summary_results:
    print(f"Capability: {r['capability']}")
    print(f"  Total Alternation Pairs: {r['total_alternation_pairs']}")
    print(f"  Failures in Alternation Pairs: {r['failures_in_alternation_pairs']}")
    print(f"  Failure Rate: {r['failure_rate_percent']}")
    print("-" * 30)


--- Overall Summary of All Capability Tests ---
Capability: Active/Passive Voice Alternation
  Total Alternation Pairs: 10
  Failures in Alternation Pairs: 9
  Failure Rate: 90.00%
------------------------------
Capability: Dative Alternation (Ditransitive Verbs)
  Total Alternation Pairs: 10
  Failures in Alternation Pairs: 9
  Failure Rate: 90.00%
------------------------------
Capability: Locative Alternation
  Total Alternation Pairs: 10
  Failures in Alternation Pairs: 10
  Failure Rate: 100.00%
------------------------------
Capability: Causative/Inchoative Alternation
  Total Alternation Pairs: 10
  Failures in Alternation Pairs: 10
  Failure Rate: 100.00%
------------------------------
Capability: Temporal Adverbial Fronting/Backing (ARGM-TMP)
  Total Alternation Pairs: 10
  Failures in Alternation Pairs: 10
  Failure Rate: 100.00%
------------------------------


In [24]:
sentence = ['Does', 'anybody', 'use', 'it', 'for', 'anything', 'else', '?'] 
predicate_id = [0, 0, 1, 0, 0, 0, 0, 0]
predicted_roles = perform_srl(sentence, predicate_id, model, vectorizer, label_encoder, scaler)
print("Predicted Roles:", predicted_roles)

Predicted Roles: ['O' 'O' 'V' 'ARG1' 'O' 'ARG2' 'O' 'O']
